In [1]:
#pip install vaderSentiment
#pip install emoji

#pip install autocorrect

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import re

import emoji

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet,stopwords
from nltk import word_tokenize,sent_tokenize
import nltk
from nltk.tokenize import regexp_tokenize 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from autocorrect import Speller
import spacy

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [3]:
nltk.download('words')
spell = Speller(lang='en')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

spacy_nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
slangs = {}
slangs_df = pd.read_csv('/content/drive/My Drive/ColabDataset/SentimentAndStockStudy/AllSlangs.csv')
slangs_df.drop('Unnamed: 0',1,inplace=True)
slangs_df.dropna(inplace=True)

In [5]:
for index,rows in slangs_df.iterrows():slangs[str(rows['Abbreviation'].replace(' ',''))] = str(rows['FullForm'].lower()).strip(' ')

In [6]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"n't":'not' 
}

In [7]:
# Reading Excel Sheet from folder
#/content/drive/My Drive/ColabDataset/SentimentAndStockStudy/TawaReview.xlsx
TawaReview = pd.read_excel("/content/drive/My Drive/ColabDataset/SentimentAndStockStudy/TawaReview.xlsx","AllReview")

In [8]:
TawaReview.head(10)

,User Rating,User Review,User Name,Review Date
0,NaN,NaN,NaN,NaN
1,5.0,Superb,Shaik Allahvali,7days ago
2,4.0,Good one,B. Sheshidhar Reddy,22days ago
3,5.0,Good Colites,Suman karan,25days ago
4,3.0,Bad product do not buy,Aswin Adarsh,1month ago
5,5.0,Nice product and good quality,Flipkart Customer,1month ago
6,3.0,It's slightly defect,Flipkart Customer,1month ago
7,3.0,Good tawa,Sk sujat Sujat,1month ago
8,4.0,Worth price,Flipkart Customer,5months ago
9,5.0,Very good product,MOHAN BABU,5months ago


In [9]:
class PreprocessingData:

  def __init__(self,df,df_name='',location=''):
    self.df = df
    self.df.dropna(inplace=True)
    self.df_name = df_name
    self.location = location


  # ----------------------------- Change Raw date to Pandas Date Time foramt -----------------------------------------
  def ago_do_date(self,ago):
    j = re.search(r'(\d+)(\w+) ago',ago)
    if(j):
      value = j.group(1)
      unit = j.group(2)
      if(not unit.endswith("s")):
        unit+='s'
      delta = relativedelta(**{unit: int(value)})
      date = datetime.now() - delta
      return date.date()
    elif(ago=='Today'):
      return datetime.now().date()
    elif(ago!='Today'):
      dt = parse(ago.replace(",",''))
      return dt.date()

  

  # ----------------------------- Removing Emojis From Raw Text -----------------------------------------
  def removingEmojis(self,reviews):
    return ''.join(rev for rev in reviews if rev not in emoji.UNICODE_EMOJI)



  # ----------------------------- Removing Slangs from Raw Text -----------------------------------------
  def removingSlangs(self,reviews):
    doc = regexp_tokenize(reviews, "[\w']+") 

    for token in doc:
      if(token in CONTRACTION_MAP):
        reviews = reviews.replace(token,CONTRACTION_MAP[token])
      elif(token in slangs):
        reviews = reviews.replace(token,slangs[token])
    return reviews

  
  
  # ----------------------------- Checking Non-English Words and Replacing Miss-Pronounce Word With Actuall Words  -----------------------------------------
  def checkingNonEnglishWords(self,reviews):
    words = set(nltk.corpus.words.words())
    for w in regexp_tokenize(reviews, "[\w']+"):
      if ((w.lower() not in words) and (w.lower()!= 'flipkart')):
        reviews = reviews.replace(w,spell(w))
    return reviews


  
  # ----------------------------- Part of Speech Tagging -----------------------------------------
  def get_wordnet_pos(self,pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

        
  
  # ----------------------------- Removing StopWords and Lemmatizing Words -----------------------------------------
  def CleaningData(self,val):
    rex = re.sub(r'[^a-zA-Z0-9]+',' ',val)

    pos = pos_tag(word_tokenize(rex))

    filter = [WordNetLemmatizer().lemmatize(x[0],PreprocessingData.get_wordnet_pos(self,x[1])) for x in pos if x[0] not in stopwords.words('english')]

    filter = ' '.join(filter)

    return filter

    
  
  # ----------------------------- Getting the Polarity of Text(both Clean and Raw) -----------------------------------------
  def sentiment_analyzer_scores(self,sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    if(score['compound']>=0.2):
      return 'pos'
    elif(score['compound']<=-0.2):
      return 'neg'
    else:
      return 'neu'
  

  def processRawData(self):
    #Date
    self.df['DateOfReview'] = self.df['Review Date'].apply(lambda x :PreprocessingData.ago_do_date(self,x))

    
    #Emojis
    self.df['User Review'] = self.df['User Review'].apply(lambda x :PreprocessingData.removingEmojis(self,x))

    #Removing Slangs
    self.df['User Review'] = self.df['User Review'].apply(lambda x :PreprocessingData.removingSlangs(self,x))

    #checkingNonEnglishWords
    self.df['User Review'] = self.df['User Review'].apply(lambda x :PreprocessingData.checkingNonEnglishWords(self,x))

    #CleaningData
    self.df['FinalText'] = self.df['User Review'].apply(lambda x :PreprocessingData.CleaningData(self,x))
    

    #sentiment_analyzer_scores
    self.df['FinalTextSentiment'] = self.df['FinalText'].apply(lambda x :PreprocessingData.sentiment_analyzer_scores(self,x))
    self.df['UserReviewSentiment'] = self.df['User Review'].apply(lambda x :PreprocessingData.sentiment_analyzer_scores(self,x))
    
    self.df.to_csv(str(fileLocation+excelSheetName+'.csv'),index=False)



fileLocation = '/content/drive/My Drive/ColabDataset/SentimentAndStockStudy/PreProcessDataset/'
excelSheetName = "TawaReview"
dataFrame = TawaReview


mj = PreprocessingData(dataFrame ,excelSheetName,fileLocation)
mj.processRawData()



TawaReview

,User Rating,User Review,User Name,Review Date,DateOfReview,FinalText,FinalTextSentiment,UserReviewSentiment
1,5.0,Superb,Shaik Allahvali,7days ago,2020-07-26,Superb,pos,pos
2,4.0,Good one,B. Sheshidhar Reddy,22days ago,2020-07-11,Good one,pos,pos
3,5.0,Good Oolites,Suman karan,25days ago,2020-07-08,Good Oolites,pos,pos
4,3.0,Bad product do not buy,Aswin Adarsh,1month ago,2020-07-02,Bad product buy,neg,neg
5,5.0,Nice product and good quality,Flipkart Customer,1month ago,2020-07-02,Nice product good quality,pos,pos
...,...,...,...,...,...,...,...,...
381,5.0,Very smart .... Worth money.... Original brand...,Thinesh kumar,4months ago,2020-04-02,Very smart Worth money Original brand And deli...,pos,pos
382,4.0,Good product in this price.,Pravat Parida,4months ago,2020-04-02,Good product price,pos,pos
383,5.0,Best product,SHASHI Yadav,4months ago,2020-04-02,Best product,pos,pos
384,5.0,Good product,Flipkart Customer,4months ago,2020-04-02,Good product,pos,pos
